### tag name:

camera_box, camera_calibration, camera_hkp, camera_image, camera_segmentation, camera_to_lidar_box_association

lidar, lidar_box, lidar_calibration, lidar_camera_projection, lidar_camera_synced_box, lidar_hkp, lidar_pose, lidar_segmentation

projected_lidar_box, stats, vehicle_pose

#### lidar
[LiDARComponent].range_image_return1

[64, 2650, 4]

channel 0: range (see spherical coordinate system definition)

channel 1: lidar intensity

channel 2: lidar elongation

channel 3: is_in_nlz (1 = in, -1 = not in)

#### lidar_camera_projection
[LiDARCameraProjectionComponent].range_image_return1

[6, 2650, 6]

channel 0: camera name

channel 1: x (axis along image width)

channel 2: y (axis along image height)

channel 3: camera name of 2nd projection (set to UNKNOWN if no projection)

channel 4: x (axis along image width)

channel 5: y (axis along image height)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
import tensorflow as tf
from waymo_open_dataset.utils import range_image_utils

2025-09-05 00:06:39.589847: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 00:06:39.591036: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-05 00:06:39.613956: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-05 00:06:39.615029: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-05 00:06:40.082118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
WAYMO_TO_SEMKITTI_20 = {
    0: 0,   # UNKNOWN -> unlabeled
    1: 1,   # CAR -> car
    2: 4,   # TRUCK -> truck
    3: 4,   # BUS -> truck
    4: 5,   # OTHER_VEHICLE -> other-vehicle
    5: 8,   # MOTORCYCLIST -> motorcyclist
    6: 7,   # BICYCLIST -> bicyclist
    7: 6,   # PEDESTRIAN -> person
    8: 19,  # SIGN -> traffic-sign
    9: 19,  # TRAFFIC_LIGHT -> traffic-sign
    10: 18, # POLE -> pole
    11: 18, # CONSTRUCTION_CONE -> pole (approx)
    12: 2,  # BICYCLE -> bicycle
    13: 3,  # MOTORCYCLE -> motorcycle
    14: 13, # BUILDING -> building
    15: 15, # VEGETATION -> vegetation
    16: 16, # TREE_TRUNK -> trunk
    17: 12, # CURB -> other-ground (alt: road=9)
    18: 9,  # ROAD -> road
    19: 9,  # LANE_MARKER -> road
    20: 12, # OTHER_GROUND -> other-ground
    21: 11, # WALKABLE -> sidewalk
    22: 11, # SIDEWALK -> sidewalk
}

def decode_jpeg_image(img_proto):
    """Decode a Waymo camera image proto into uint8 HxWx3 (RGB)."""
    img = tf.image.decode_jpeg(img_proto, channels=3)
    return img.numpy()

# def decode_jpeg_image(img_proto):
#     img_array = np.frombuffer(img_proto, np.uint8)
#     img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)  # Returns BGR
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)       # Convert to RGB
#     return img

def _sample_rgb_nearest(img, rows, cols):
    h, w = img.shape[:2]
    rows = np.clip(rows, 0, h - 1)
    cols = np.clip(cols, 0, w - 1)
    return img[rows, cols, :]

def colorize_range_image_v2(
    proj,                 # [H, W, 6] = [cam1,row1,col1, cam2,row2,col2]
    camera_images,        # dict: {camera_id:int -> np.uint8 [Hc,Wc,3]}
    default_color=(128,128,128)
):
    """
    Returns:
      colors_hw3: uint8 [H, W, 3]
      used_mask:  bool  [H, W]  (True where a camera projection was used)
    """
    H, W, C = proj.shape
    assert C == 6, "Expected v2 projection with two triplets: [cam1,row1,col1, cam2,row2,col2]"
    colors = np.empty((H, W, 3), dtype=np.uint8)
    colors[:] = np.array(default_color, dtype=np.uint8)
    used = np.zeros((H, W), dtype=bool)

    cam1 = proj[..., 0]; r1 = proj[..., 1]; c1 = proj[..., 2]
    cam2 = proj[..., 3]; r2 = proj[..., 4]; c2 = proj[..., 5]
    # SWAP row and col here!
    r1, c1 = c1, r1
    r2, c2 = c2, r2
    # Pass 1: try primary projection per camera
    for cid, img in camera_images.items():
        m1 = (cam1 == cid)
        if not np.any(m1):
            continue
        rr = r1[m1].astype(np.int32)
        cc = c1[m1].astype(np.int32)
        rgb = _sample_rgb_nearest(img, rr, cc)
        ii, jj = np.nonzero(m1)
        colors[ii, jj, :] = rgb
        used[ii, jj] = True

    # Pass 2: fallback to secondary where primary failed
    need = ~used
    for cid, img in camera_images.items():
        m2 = (cam2 == cid) & need
        if not np.any(m2):
            continue
        rr = r2[m2].astype(np.int32)
        cc = c2[m2].astype(np.int32)
        rgb = _sample_rgb_nearest(img, rr, cc)
        ii, jj = np.nonzero(m2)
        colors[ii, jj, :] = rgb
        used[ii, jj] = True

    return colors, used

def colorize_waymo(label_channel, lidar_channel, extrinsic, beam_inclination, lidar_camera_projection, cam_images):
    # Get the semantic ID (ignore instance ID)
    semantic_id = label_channel[:,:,1]
    # Convert orig_sem_label to sem_label using the map
    sem_label = np.vectorize(WAYMO_TO_SEMKITTI_20.get)(semantic_id)
    # Get x, y, z from lidar_channel
    lidar_range = lidar_channel[:,:,0]
    lidar_intensity = lidar_channel[:,:,1]

    # #########################################################
    # convert lidar_range, extrinsic, and beam_inclination to tensors with [B, W, H] with B=1
    lidar_range = tf.convert_to_tensor(lidar_range, dtype=tf.float32)
    lidar_range = tf.expand_dims(lidar_range, axis=0)
    extrinsic = tf.convert_to_tensor(extrinsic, dtype=tf.float32)
    extrinsic = tf.expand_dims(extrinsic, axis=0)
    beam_inclination = tf.convert_to_tensor(beam_inclination, dtype=tf.float32)
    beam_inclination = tf.expand_dims(beam_inclination, axis=0)
    # Convert to 3D points (vehicle frame)
    points = range_image_utils.extract_point_cloud_from_range_image(lidar_range, extrinsic, beam_inclination)
    #  make point to by numarray
    points_np = points.numpy()
    # squeeze to remove dimensions of size 1
    points_np = points_np.squeeze()
    lidar_range = lidar_range.numpy().squeeze()
    # sanity check points_np: if lidar_range = -1, then points_np = -1
    points_np[lidar_range == -1] = 0
    #  put the range image so that it has size {H, W} with five channel: range, x, y, z, intensity in such order
    # lidar_raw[:,:,0] then points_np then lidar_raw[:,:,1]
    points_final = np.zeros((64, 2650, 5))
    points_final[:,:,0] = lidar_range
    points_final[:,:,1:4] = points_np
    points_final[:,:,4] = lidar_intensity
    # now we have data that similar to that of the paper. Just need to add R,G, B (which is huge thing to do)
    rgb_grid, used_mask = colorize_range_image_v2(lidar_camera_projection, cam_images, default_color=(0,0,0))
    # combine the final points with range, x, y, z, intensity, flag, R, G, B and label
    # flag is valid when and only when range > 0 and used_mask is one
    points_final_output = np.zeros((64, 2650, 10))
    points_final_output[:,:,0] = points_final[:,:,0]  # range
    points_final_output[:,:,1] = points_final[:,:,1]  # x
    points_final_output[:,:,2] = points_final[:,:,2]  # y
    points_final_output[:,:,3] = points_final[:,:,3]  # z
    points_final_output[:,:,4] = points_final[:,:,4]  # intensity
    points_final_output[:,:,5] = (points_final[:,:,0] > 0) & (used_mask)  # flag
    points_final_output[:,:,6] = rgb_grid[:,:, 0]  # R
    points_final_output[:,:,7] = rgb_grid[:,:, 1]  # G
    points_final_output[:,:,8] = rgb_grid[:,:, 2]  # B
    points_final_output[:,:,9] = sem_label  # label (not used)

    # sanity put: any -1 element is set to 0
    points_final_output[points_final_output == -1] = 0

    return points_final_output  # [H, W, 10]: range, x, y, z, intensity, flag, R, G, B, label

In [3]:
# Path to the directory with all components
dataset_dir = '../../WoD/validation'
height = 64
width = 2650
preprocess_dir = f'{dataset_dir}/preprocess'
os.makedirs(preprocess_dir, exist_ok=True)
# search all .parquet files in the lidar_segmentation directory
lidar_segmentation_dir = f'{dataset_dir}/lidar_segmentation'
context_names = sorted([f for f in os.listdir(lidar_segmentation_dir) if f.endswith('.parquet')])
# go through the list of context_names, make sure corresponding files exist in lidar, camera_image, lidar_calibration, and lidar_camera_projection
for context_name in context_names:
    lidar_path = f'{dataset_dir}/lidar/{context_name}'
    camera_path = f'{dataset_dir}/camera_image/{context_name}'
    lidar_calibration_path = f'{dataset_dir}/lidar_calibration/{context_name}'
    lidar_camera_projection_path = f'{dataset_dir}/lidar_camera_projection/{context_name}'
    if not (os.path.exists(lidar_path) and os.path.exists(camera_path) and os.path.exists(lidar_calibration_path) and os.path.exists(lidar_camera_projection_path)):
        print(f'Missing files for context: {context_name}')

In [4]:
# lidar_segmentation_df = pd.read_parquet(f'{dataset_dir}/lidar_segmentation/{context_name}')
# lidar_df = pd.read_parquet(f'{dataset_dir}/lidar/{context_name}')
# camera_df = pd.read_parquet(f'{dataset_dir}/camera_image/{context_name}')
# lidar_calibration_df = pd.read_parquet(f'{dataset_dir}/lidar_calibration/{context_name}')
# lidar_camera_projection_df = pd.read_parquet(f'{dataset_dir}/lidar_camera_projection/{context_name}')

In [5]:
# for frame_index, frame_data in tqdm(lidar_segmentation_df.iterrows(), desc="Processing Frames"):
#     timestamp = frame_data.get('key.frame_timestamp_micros', None)
#     label_channel = lidar_segmentation_df[lidar_segmentation_df['key.frame_timestamp_micros'] == timestamp]['[LiDARSegmentationLabelComponent].range_image_return1.values']
#     label_channel = np.array(label_channel.values[0]).reshape(height, width, 2) 
#     lidar_channel = lidar_df[(lidar_df['key.frame_timestamp_micros'] == timestamp) & (lidar_df['key.laser_name'] == 1)]['[LiDARComponent].range_image_return1.values']
#     lidar_channel = np.array(lidar_channel.values[0]).reshape(height, width, 4) 
#     lidar_calibration = lidar_calibration_df[lidar_calibration_df['key.laser_name'] == 1]
#     extrinsic = lidar_calibration['[LiDARCalibrationComponent].extrinsic.transform'].values[0].reshape(4, 4)
#     beam_inclination = lidar_calibration['[LiDARCalibrationComponent].beam_inclination.values'].values[0]
#     lidar_camera_projection = lidar_camera_projection_df[(lidar_camera_projection_df['key.frame_timestamp_micros'] == timestamp) & (lidar_camera_projection_df['key.laser_name'] == 1)]['[LiDARCameraProjectionComponent].range_image_return1.values']
#     lidar_camera_projection = np.array(lidar_camera_projection.values[0]).reshape(height, width, 6)
#     cameras = camera_df[camera_df['key.frame_timestamp_micros'] == timestamp]
#     # go through each row in cameras, decode the image with [CameraImageComponent].image and key.camera_name (not that camera ID = key.camera_name - 1)
#     cam_images = {}
#     for index, row in cameras.iterrows():
#         cam_images[row['key.camera_name']] = decode_jpeg_image(row['[CameraImageComponent].image'])
#     pcd = colorize_waymo(label_channel, lidar_channel, extrinsic, beam_inclination, lidar_camera_projection, cam_images)
#     # save pcd to file
#     output_file = os.path.join(preprocess_dir, context_name.replace('.parquet', f'_{timestamp}.bin'))
#     pcd.astype(np.float32).tofile(output_file)


In [6]:
for context_name in tqdm(context_names, desc="Processing Contexts"):
    # Read the components
    lidar_segmentation_df = pd.read_parquet(f'{dataset_dir}/lidar_segmentation/{context_name}')
    lidar_df = pd.read_parquet(f'{dataset_dir}/lidar/{context_name}')
    camera_df = pd.read_parquet(f'{dataset_dir}/camera_image/{context_name}')
    lidar_calibration_df = pd.read_parquet(f'{dataset_dir}/lidar_calibration/{context_name}')
    lidar_camera_projection_df = pd.read_parquet(f'{dataset_dir}/lidar_camera_projection/{context_name}')
    # lidar calibration is per context
    lidar_calibration = lidar_calibration_df[lidar_calibration_df['key.laser_name'] == 1]
    extrinsic = lidar_calibration['[LiDARCalibrationComponent].extrinsic.transform'].values[0].reshape(4, 4)
    beam_inclination = lidar_calibration['[LiDARCalibrationComponent].beam_inclination.values'].values[0]

    # Go through each frame in the context
    for frame_index, frame_data in tqdm(lidar_segmentation_df.iterrows(), desc="Processing Frames"):
        timestamp = frame_data.get('key.frame_timestamp_micros', None)
        # from now on:
        # label from lidar_segmentation
        # lidar data from lidar and lidar_camera_projection
        # camera data from camera
        # all filtered by timestamp
        label_channel = lidar_segmentation_df[lidar_segmentation_df['key.frame_timestamp_micros'] == timestamp]['[LiDARSegmentationLabelComponent].range_image_return1.values']
        label_channel = np.array(label_channel.values[0]).reshape(height, width, 2)  
        lidar_channel = lidar_df[(lidar_df['key.frame_timestamp_micros'] == timestamp) & (lidar_df['key.laser_name'] == 1)]['[LiDARComponent].range_image_return1.values']
        lidar_channel = np.array(lidar_channel.values[0]).reshape(height, width, 4)  
        lidar_camera_projection = lidar_camera_projection_df[(lidar_camera_projection_df['key.frame_timestamp_micros'] == timestamp) & (lidar_camera_projection_df['key.laser_name'] == 1)]['[LiDARCameraProjectionComponent].range_image_return1.values']
        lidar_camera_projection = np.array(lidar_camera_projection.values[0]).reshape(height, width, 6)
        cameras = camera_df[camera_df['key.frame_timestamp_micros'] == timestamp]
        # go through each row in cameras, decode the image with [CameraImageComponent].image and key.camera_name
        cam_images = {}
        for index, row in cameras.iterrows():
            cam_images[row['key.camera_name']] = decode_jpeg_image(row['[CameraImageComponent].image'])
        pcd = colorize_waymo(label_channel, lidar_channel, extrinsic, beam_inclination, lidar_camera_projection, cam_images)
        # save pcd to file
        output_file = os.path.join(preprocess_dir, context_name.replace('.parquet', f'_{timestamp}.bin'))
        pcd.astype(np.float32).tofile(output_file)



Processing Contexts:   0%|          | 0/202 [00:00<?, ?it/s]

Processing Frames: 0it [00:00, ?it/s]

2025-09-05 00:06:54.646552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-09-05 00:06:54.646836: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Processing Frames: 0it [00:00, ?it/s]

Processing Frames: 0it [00:00, ?it/s]

...

Processing Frames: 0it [00:00, ?it/s]

Processing Frames: 0it [00:00, ?it/s]

Processing Frames: 0it [00:00, ?it/s]

Processing Frames: 0it [00:00, ?it/s]

Processing Frames: 0it [00:00, ?it/s]